# ΗΥ360: Δεύτερο Μέρος Συνθετικής Εργασίας
## Τμήμα 2 | Εξερεύνηση του GitHub Dataset μέσω του Colaboratory (30 μονάδες)
---

### Υποδείξεις - διαβάστε τις πολύ προσεκτικά! - :

* Σιγουρευτείτε ότι διαβάσατε καλά τις οδηγίες σε κάθε κελί και κατανοήσατε τι υλοποιεί πριν το εκτελέσετε.
* Να θυμάστε ότι έχετε τη δυνατότητα να μεταφορτώνετε το αρχικό "σημειωματάριο" όποτε το χρειάζεστε.
* Μπορείτε να δημιουργείτε νέα κελιά για να τα χρησιμοποιείτε σε ελέγχους, εκσφαλμάτωση, εξερεύνηση κλπ. Μάλιστα προτείνουμε να το κάνετε! 
**Βεβαιωθείτε εντούτοις ότι η τελική απάντηση σε κάθε ερώτηση βρίσκεται στο _δικό της κελί_ και _προσδιορίζεται ρητά_**.
* Το Colaboratory δεν σας ειδοποιεί για τα bytes που θα καταναλώσει η εκτέλεση των SQL ερωτημάτων σας.  **Σιγουρευτείτε ότι ελέγχετε την κατανάλωση μέσω της διεπαφής (UI) του BigQuery πριν εκτελέσετε τα ερωτήματά σας στο Colaboratory!**
* Ακολουθείστε τις οδηγίες υποβολής.

## Μέλη της Ομάδας Εργασίας:
Παραθέστε τα ονοματεπώνυμα και τους ΑΜ των μελών της ομάδας στην ακόλουθη λίστα:

*   Καραθανάσης Δημητρης 3547
*   Συλλιγάρδος Εμμανουήλ 3849



## Ρυθμίσεις για το BigQuery και τις σχετικές εξαρτήσεις

Εκτελέστε τα δύο ακόλουθα κελιά (shift + enter) προκειμένου να πιστοποιήσετε την εργασία σας και να φορτώσετε τις απαιτούμενες βιβλιοθήκες. 

Προσέξτε ότι θα χρειαστεί να συμπληρώσετε τη μεταβλητή `project_id` στο πρώτο κελί με το Google Cloud project ID που έχετε δημιουργήσει για τις ανάγκες της εργασίας σας.  Για να δείτε το project ID μεταβείτε στη σελίδα https://console.cloud.google.com/cloud-resource-manager.



In [0]:
# Εκτελέστε αυτό το κελί προκειμένου να πιστοποιήσετε την εργασία σας στο BigQuery.
from google.colab import auth
auth.authenticate_user()
project_id = "ancient-tractor-226920"

In [0]:
# Βιβλιοθήκες που θα χρειαστείτε
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import altair as alt
import pandas as pd

%matplotlib inline
plt.style.use('seaborn-whitegrid')




# Σχετικά

Το BigQuery διαθέτει ένα τεράστιο σύνολο δεδομένων (dataset) με αρχεία και στατιστικά από το GitHub  που περιέχουν πληροφορία σχετική με αποθετήρια (repositories), δεσμεύσεις (commits) και περιεχόμενα αρχείων.  Στο τμήμα αυτό της εργασίας σας θα εντρυφήσουμε σε αυτό το σύνολο δεδομένων. Μην ανησυχείτε εάν δεν είσαστε εξοικειωμένοι με τα Gits και το GitHub -- θα εξηγηθεί επαρκώς ότι χρειάζεστε για να ολοκληρώσετε αυτό το τμήμα εργασίας.

---
## Σημειώσεις

** Το σύνολο δεδομένων του GitHub που έχει αποθηκευτεί στο BigQuery είναι τεράστιο. *Μια και μόνη επερώτηση μόνο στον πίνακα "contents"  (που έχει μέγεθος 2.16TB!) μπορεί να καταναλώσει τη δωρεάν μηνιαία χρήση του 1TB που παρέχεται σε όλους τους χρήστες και ακόμη περισσότερο.***

Για να γίνει περισσότερο διαχειρίσιμο αυτό το τμήμα της εργασίας έχουμε φτιάξει ένα υποσύνολο των πρωτότυπων δεδομένων. Διατηρήσαμε σχεδόν ολόκληρη την πληροφορία από τους πρωτότυπους πίνακες, αλλά επιλέξαμε να περιοριστούμε στα  500,000 αποθετήρια (repositories) του GitHub στα οποία έγιναν οι περισσότερες προσπελάσεις ανάμεσα στον Ιανουάριο του 2016 και τον Οκτώβριο του 2018.
Οι πίνακες με τους οποίους θα εργαστούμε βρίσκονται [εδώ](https://console.cloud.google.com/bigquery?utm_source=bqui&utm_medium=link&utm_campaign=classic&project=cs360nt&pli=1&p=cs360nt&d=project_part_2_2&page=dataset). 

### Εντούτοις, για να μπορέσετε να τους προσπελάσετε και να θέσετε ερωτήματα, απατείται να γίνει εκ μέρους σας έγκαιρα μια ενέργεια: θα πρέπει να κατανοήσετε τι χρειάζεται να ρυθμιστεί και το αργότερο μέχρι 11/1/2019 να έχετε επικοινωνήσει στο email: tsatsaki@csd.uoc.gr ζητώντας να γίνει η κατάλληλη ρύθμιση. Επαναλαμβάνουμε: θα πρέπει να κατανοήσετε σε τι αφορά η ρύθμιση και να υποβάλλετε τα απαραίτητα στοιχεία για να γίνει.

**Αφού εξασφαλίσετε τη δυνατότητα προσπέλασης, περιηγηθείτε στα σχήματα των πινάκων για να κατανοήσετε τα δεδομένα που περιέχουν**.  Να σημειωθεί ότι και σε αυτό το σύνολο δεδομένων υπάρχουν πίνακες πολύ μεγάλοι (ο πίνακας contents έχει μέγεθος μεγαλύτερο από 500GB), γι' αυτό θα πρέπει να προσέχετε πώς τους χρησιμοποιείτε.  Ελέγξτε τη χρέωση για τα ερωτήματα που θα θέσετε, **πριν τα θέσετε,** στη διεπαφή του BigQuery.

---

## Ένας πολύ σύντομος οδηγός στο GitHub

Εάν δεν είσαστε εξοικειωμένοι με τα Git και το GitHub, ακολουθούν αδρές επεξηγήσεις των βασικών εννοιών που πλαισιώνουν αυτό το τμήμα της εργασίας:

*  *GitHub*: Το GitHub είναι πάροχος υπηρεσίας ελέγχου εκδόσεων αρχείων, που επιτρέπει (μεταξύ άλλων) τη συνεργατική υλοποίθηση και παρακολούθηση πηγαίου κώδικα, με αρκετά αποτελεσματικό τρόπο.

*  *commit*: Ως commit (αναθεώρηση) θεωρείται η αλλαγή που εφαρμόζεται σε ένα σύνολο αρχείων. Έτσι, εάν κάνετε αλλαγές σε ένα σύνολο αρχείων που είναι σε μια κατάσταση Α, μετά από *commit* το σύνολο θα βρίσκεται σε μια νέα κατάσταση Β. Ένα commit χαρακτηρίζεται από ένα "μίγμα" (*hash ή SHA*) πληροφοριών που σχετίζονται με την αναθεώρηση (τον συντάκτη του commit, ποιος στην πραγματικότητα έκανε  [εφάρμοσε] τις αλλαγές στα αρχεία, σε τι αφορούν οι αλλαγές, κλπ.)

* *parent commit*: Το commit που έγινε πριν από το τρέχον commit.

*   *repo*: Αποθετήριο ονομάζεται μια αφηρημένη συλλογή (κάτι σαν φάκελος) αρχείων μαζί με ένα ιστορικό αναθεωρήσεων (commits) αυτών. Εάν το GitHub username σας είναι "foo" και φτιάξετε ένα αποθετήριο (repo) με όνομα "data-rocks", το απόλυτο όνομά του θα είναι "foo/data-rocks".  Μπορείτε να σκέφτεστε την ιστορία των αποθετηρίων σε σχέση με τις αναθεωρήσεις τους (commits).  Πχ το "foo/data-rocks" μπορεί να πέρασε από ένα σύνολο "καταστάσεων" A->B->C->D, στο οποίο κάθε αλλαγή κατάστασης (A->B, B->C, C->D) σχετίζεται με μια αναθεώρηση (commit).

*   *branch*:  Προκειμένου να παρακολουθήσουν διαφορετικά ιστορικά αναθεωρήσεων, τα αποθετήρια του GitHub μπορεί να έχουν διακλαδώσεις.  Η 'κύρια' διακλάδωση ενός αποθετήριου ονομάζεται 'master' branch.  Έστω ότι στο "foo/data-rocks" έχουμε την ιστορία αναθεωρήσεων  A->B->C->D στην κύρια διακλάδωση. Εάν κάποιος αποφασίσει να προσθέσει ένα νέο χαρακτηριστικό  στο "foo/data-rocks",  μπορεί να δημιουργήσει μια διακλάδωση με όνομα"cool-new-feature" που ξεχωρίζει από την κύρια διακλάδωση. Όλος ο κώδικας της κύριας διακλάδωσης θα βρίσκεται στην καινούργια, αλλά ο κώδικας που θα προστεθεί στην καινούργια δεν θα υπάρχει στην κύρια διακλάδωση (από την οποία προέκυψε η καινούργια διακλάδωση).

*   *ref*: Για το σκοπό αυτού του τμήματος της εργασίας, μπορείτε να θεωρείτε το πεδίο 'ref' του πίνακα "αρχείων" ως αυτό που αναφέρεται στη διακλάδωση στην οποία "κατοικεί" το αρχείο σ' ένα αποθετήριο σε μια δεδομένη στιγμή.



Για το τμήμα αυτό της εργασίας δεν χρειάζεται να γνωρίζετε με λεπτομέρεια τα ακόλουθα:


*   Δέντρα αναθεωρήσεων (commit trees)
*   Το γνώρισμα κωδικοποίησης (encoding attribute) του πίνακα αναθεωρήσεων

Για περισσότερες πληροφορίες μπορείτε να δείτε [εδώ](https://ellak.gr/wiki/index.php?title=%CE%9F%CE%B4%CE%B7%CE%B3%CE%AF%CE%B5%CF%82_%CF%87%CF%81%CE%AE%CF%83%CE%B7%CF%82_%CF%84%CE%BF%CF%85_Github) και [εδώ](https://help.github.com/).

# Ενότητα 1 | Γνωριμία με τα δεδομένα του GitHub
---

## Κατανόηση των πινάκων του GitHub

Γνωρίζουμε ότι τα διαγράμματα οντοτήτων-συσχετίσεων είναι μία αναπαράσταση της δομής μιας ΒΔ (συνόλου πινάκων) με μη τεχνικό τρόπο και με όλη την απαραίτητη πληροφορία για τη ΒΔ. Όπως θα φανεί, με τα διαγράμματα οντοτήτων-συσχετίσεων θα εξοικειωθούμε με τους πίνακες του συνόλου δεδομένων GitHub, πριν ακόμη αναλύσουμε τα δεδομένα τους.

## Ερώτηση 1: Πίνακες CS360 GitHub --> Διάγραμμα οντοτήτων-συσχετίσεων (4 μονάδες)

Δημιουργήστε ένα διάγραμμα οντοτήτων-συσχετίσεων για τα δεδομένα που περιέχονται στους πίνακες `cs360nt:project_part_2_2` που βρίσκονται [εδώ](https://console.cloud.google.com/bigquery?utm_source=bqui&utm_medium=link&utm_campaign=classic&project=cs360nt&pli=1&p=cs360nt&d=project_part_2_2&page=dataset).  Αγνοήστε τον πίνακα  `github_repo_readme_contents_cs360` (θα τον χρησιμοποιήσετε σε επόμενα ερωτήματα).

### Σημειώσεις

* Είναι πιθανό να μην είναι δυνατή η απευθείας "μετάφραση" των πινάκων του CS360 GitHub Repo σε διάγραμμα οντοτήτων-συσχετίσεων με τον τρόπο που γνωρίζετε από το μάθημα.  Σημαντικό μέρος αυτής της ερώτησης είναι η ανάλυση των πινάκων, η σκέψη και ο προσδιορισμός των σχέσεων μεταξύ των αντικειμένων που περιέχουν και η δημιουργία εν τέλει ενός εύλογου διαγράμματος οντοτήτων-συσχετίσεων βασισμένου στην ανάλυση που θα κάνετε.

* Θεωρήστε τα γνωρίσματα "author" και "committer" που έχουν τύπο εγγραφής (record) ως μοναδιαία (με απλό τύπο).  Είναι σημαντικό ότι δε χρειάζεται να συμπεριλάβετε τα `committer.name`, `committer.email`, κλπ  στα διαγράμματά σας.  Να σημειωθεί ότι το γνώρισμα "language" έχει τύπο *array*, γεγονός που πρέπει να ληφθεί υπόψη στη σχεδίαση των διαγραμμάτων σας.

* Τα διαγράμματά σας πρέπει να είναι αρχεία εικόνας που θα σχεδιάσετε με όποιον τρόπο θέλετε (κατάλληλο λογισμικό ή με το χέρι), **αρκεί να είναι ευανάγνωστα**. Θα τα συμπεριλάβετε στο σημειωματάριό σας ως εξής:



> 1. Προσθέστε το αρχείο εικόνας στο Google Drive σας 
> 2. Δημιουργήστε ένα κοινόχρηστο URL για το αρχείο σας, και σημειώστε το πεδίο "ID" από το URL που θα δημιουργηθεί
 > Το URL θα έχει τη μορφή "`https://drive.google.com/open?id=<some ID>`"
> 3. Προσθέστε την ακόλουθη επισήμανση (markup) στο κατάλληλο κελί του σημειωματαρίου σας l `![](https://drive.google.com/uc?export=view&id=<your image ID>)`
> 4. Εκτελέστε (run) τον κώδικα στο κελί σας.


---
 ![](https://drive.google.com/uc?export=view&id=1BKSdYDLQM3LIFuucpI9rFaQfBtZKYGaP)
 
---

## Ερώτηση 2: Εξηγήστε το διάγραμμα οντοτήτων-συσχετίσεων που φτιάξατε (2 μονάδες)

Σε μια μικρή παράγραφο εξηγήστε το διάγραμμά σας. Θα πρέπει να καλύψετε τουλάχιστον τα ακόλουθα:

*   ποιες είναι οι οντότητες,
*   ποιες οι μεταξύ τους σχέσεις (αναφέρετε εάν πρόκειται για 1-Ν, Ν-1, κλπ.),
*   ποια είναι τα κλειδιά σε καθεμιά οντότητα.

Πρέπει επίσης να εξηγήσετε σύντομα με ποιο τρόπο καθορίσατε τη συνολική δομή του διαγράμματός σας.

---
Οι οντότητες στο διάγραμμα μας είναι:

*   Commit
*   Licence
*   File
*   Languages
*   Content

Οι σχέσεις είναι:

*   Commit --(1, 1)--> Γίνεται <--(0, Ν)-- Repository
*   Licence --(0, Ν)--> Χρησιμοποιεί <--(1, 1)-- Repository
*   File --(1, 1)--> Βρίσκεται <--(0, N)-- Repository
*   Languages --(1, 1)--> Γράφτηκε <--(1, 1)-- Repository
*   Content --(1, 1)--> Περιέχει <--(1, 1)-- File

Τα κλειδιά κάθε οντότητας είναι:

*   Commit : commit
*   Licence : repo_name
*   File : id
*   Languages : repo_name
*   Content : id
---

## Ερώτηση 3: Μεταφράστε το διάγραμμά σας στο αντίστοιχο σχεσιακό σχήμα (3 μονάδες)

Δώστε το σχεσιακό σχήμα που αντιστοιχεί στο διάγραμμα που σχεδιάσατε στην προηγούμενη ερώτηση. Αυτό, θα πρέπει να διαφέρει από το σχήμα σύμφωνα με το οποίο φτιάχτηκαν οι πίνακες του συνόλου δεδομένων CS360 GitHub Repo. 

Σιγουρευτείτε ότι έχετε καθορίσει στο σχήμα σας:
1. το ***όνομα*** κάθε γνωρίσματος (μην αναφερθείτε σε τύπους),
2. το ***κλειδί κάθε πίνακα***,
3. ***ξένα κλειδιά σε κάθε πίνακα*** και σε ποιους πίνακες αναφέρονται. 

---
*   Repository

repo_name | license
----------------|
   
*   Commit

commit | repo_name | message | subject | trailer | commiter | difference | author | difference_truncated | parrent | encoding | tree
----------------|-----------------|-----------------

*   License

license | 
----------------|

*   File

id | repo_name | mode | path | ref | symlink_target
----------------|-----------------|-----------------

*   Languages

name | repo_name | bytes
----------------|-----------------|-----------------

*   Content

id | id_file | repo_name | copies | binary | content | size
----------------|-----------------|-----------------

---

## Ερώτηση 4: Ανάλυση (2 points)

Έχετε πλέον στη διάθεσή σας δύο σχήματα: αυτό που φτιάξατε στην ερώτηση 3 και αυτό που είχαν οι πίνακες όπως σας τους δώσαμε.

** Σε μια και μόνη παράγραφο (μέχρι 100 λέξεις), συγκρίνετέ τα. Ποιο θεωρείτε καλύτερο;**

Δεν υπάρχει μοναδική σωστή απάντηση. Τα σχήματα των ΒΔ θα πρέπει να καλύπτουν επαρκώς και τις εφαρμογές οι οποίες θα χρησιμοποιήσουν τις ΒΔ.


---
Παρατηρώντας, αρχικά τα δύο σχήματα παρατηρούμε ότι οι διαφορές δεν είναι μεγάλες. Συγκεκριμένα, στο δικό μας σχήμα έχουμε επιπλέον ένα πίνακα τον Repository. Ο πίνακας αυτός περιέχει τα ονόματα όλων των repositories του Github και το licence που χρησιμοποιεί κάθε repository. Συνεπώς, στο σχήμα μας, ο πίνακας licence περιέχει μόνο τόσα entries όσα και τα διαφορετικά licences που υπάρχουν και όχι 1 entry για κάθε repo. Ακόμα στον πίνακα Content έχουμε προσθέσει το id_file και το repo_name στο οποίο ανήκει αυτό το content. Έτσι μπορούμε να βρούμε που το κάθε content άμεσα, χωρίς join, να βρούμε σε ποιο file και σε ποιο repo ανήκει. Τελικά, οι διαφορές των δύο σχημάτων είναι λίγες, παρόλα αυτά, θεωρούμε καλύτερο το δικό μας αφού είναι πιο ξεκάθαρο και γλιτώνει μερικά join στα queries.

---

# Ενότητα 2 | Οπτικοποίηση του Git!
---

## Πριν ξεκινήσετε ...

Τώρα που έχετε κατανοήσει το σύνολο δεδομένων με το οποίο θα ασχοληθείτε, θα συνεχίσετε με την ανάλυση ορισμένων από τις ιδιότητές του. Για τις απαιτούμενες οπτικοποιήσεις μπορείτε να χρησιμοποιήσετε οποιαδήποτε βιβλιοθήκη γραφικών αναπαραστάσεων θέλετε. Προτείνουμε κάποια από τις:

*   seaborn (https://seaborn.pydata.org/tutorial.html)
*   matplotlib (https://matplotlib.org/3.0.0/tutorials/index.html)
*   altair (https://altair-viz.github.io/)
*   pandas (https://pandas.pydata.org/pandas-docs/stable/visualization.html)


> * **σημειώστε ότι**: μπορείτε, εάν θέλετε, να σχεδιάζετε μέσα από ένα [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) .





### Χρήση του BigQuery στο Collab

Τα σημειωματάρια στο Jupyter (στα οποία βασίζονται τα σημειωματάρια του Collab) χρησιμοποιούν τη ιδέα της "μαγείας".
Εάν γράψετε την ακόλουθη γραμμή στην κορυφή ενός κελιού με 'Κώδικα' :

```
%%bigquery --project $project_id variable # this is the key line
SELECT ....
FROM ...
```

το "%%" μετατρέπει το κελί σε κελί SQL. Ο πίνακας που παράγεται από το ερώτημα αποθηκεύεται στη μεταβλητή `variable`. Στη συνέχεια μπορείτε να χρησιμοποιήσετε τη μεταβλητή `variable` στη βιβλιοθήκη οπτικοποίησης που θα χρησιμοποιήσετε για να δημιουργήσετε γραφικές αναπαραστάσεις!

Εκτελέστε τα δύο ακόλουθα κελιά για να πάρετε μια ιδέα του τι γίνεται στην πράξη.

In [0]:
%%bigquery --project $project_id example

SELECT lrepo_name, watch_count
FROM `cs360nt.project_part_2_2.github_repos_cs360`
ORDER BY watch_count DESC
LIMIT 10;

In [0]:
example.head()

## Ερώτηση 5: Κατανομές τιμών για διάφορα πεδία (γνωρίσματα)  (9 μονάδες)



Ας βρέξουμε τα πόδια μας στα δεδομένα του συνόλου που μελετούμε δημιουργώντας τις ακόλουθες γραφικές παραστάσεις:

1.  Κατανομή αδειών (licences) στα διάφορα αποθετήρια (repos)
2.  Κατανομή γλωσσών (languages) στα διάφορα αποθετήρια (repos)
3.  Κατανομή μεγέθους αρχείων (file sizes) στα διάφορα αποθετήρια (repos)
4.  Κατανομή πλήθους αρχείων (files) που περιλαμβάνονται στα διάφορα αποθετήρια (repos)
5.  Αριθμός των αναθεωρήσεων (commits) κατά συντάκτη (author) και αναθεωρητή (committer) στα διάφορα αποθετήρια (repos)


Λάβετε υπόψιν ότι δεν θα πάρετε όλες τις μονάδες εάν τα διαγράμματά σας δεν είναι καλά φτιαγμένα (πχ δυσανάγνωστα).


### Συμβουλές
* Ορισμένα διαγράμματα θα χρειαστεί να έχουν τουλάχιστον ένα άξονά τους σε λογαριθμική κλίμακα (log-scaled) για να είναι ευανάγνωστα
* Για δημιουργία ευανάγνωστων διαγραμμάτων μπορείτε να χρησιμοποιήσετε [pandas.DataFrame.sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html). Δείγμα μεγέθους μεταξύ 1,000 και 10,000 θα δώσει περισσότερο ευανάγνωστα διαγράμματα.

### Να θυμάστε:
* Να είσαστε προσεκτικοί με τα ερωτήματά σας!  Μην εκτελείτε `SELECT *` τυφλά σε κάποιο πίνακα στο παρόν σημειωματάριο του Colaboratory, καθώς δεν θα λάβετε προειδοποίηση του μεγέθους των δεδομένων που θα καταναλωθούν για το ερώτημά σας. Δοκιμάστε πρώτα το ερώτημά σας στο BigQuery UI καθώς εκεί έχετε τις απαιτούμενες προειδοποιήσεις -- ακόμη καλύτερα βάλτε και όρια στα ερωτήματά σας με βάση όσα έχουμε ήδη πει. 
* Μην ξεχνάτε να χρησιμοποιείτε το υποσύνολο δεδομένων cs360nt:project_part_2_2 που βρίσκονται [εδώ](https://console.cloud.google.com/bigquery?utm_source=bqui&utm_medium=link&utm_campaign=classic&project=cs360nt&pli=1&p=cs360nt&d=project_part_2_2&page=dataset).



### a) Κατανομή αδειών (1 μονάδα)
#### (x-άξονας: είδος άδειας (license type), y-άξονας: # αποθετηρίων (repos) που περιέχουν αυτή την άδεια)


In [4]:
%%bigquery --project $project_id q5a

SELECT COUNT(repo_name) AS Nr_of_repos, license
FROM `bigquery-public-data.github_repos.licenses`
GROUP BY license

,Nr_of_repos,license
0,8696,artistic-2.0
1,17814,isc
2,1710301,mit
3,27044,cc0-1.0
4,24425,epl-1.0
5,345720,gpl-2.0
6,343693,gpl-3.0
7,18929,mpl-2.0
8,41513,agpl-3.0
9,22994,lgpl-2.1


In [12]:
alt.Chart(q5a, height = 600, width = 600).mark_bar().encode(
    x="license",
    y="Nr_of_repos"
)

Chart({
  data:     Nr_of_repos       license
  0          8696  artistic-2.0
  1         17814           isc
  2       1710301           mit
  3         27044       cc0-1.0
  4         24425       epl-1.0
  5        345720       gpl-2.0
  6        343693       gpl-3.0
  7         18929       mpl-2.0
  8         41513      agpl-3.0
  9         22994      lgpl-2.1
  10        40069      lgpl-3.0
  11        46766     unlicense
  12       493471    apache-2.0
  13        56062  bsd-2-clause
  14       153409  bsd-3-clause,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'license'
    }),
    y: Y({
      shorthand: 'Nr_of_repos'
    })
  }),
  height: 600,
  mark: 'bar',
  width: 600
})

### b) Κατανομή γλωσσών (1 μονάδα)
#### (x-άξονας: γλώσσα προγραμματισμού (programming language), y-άξονας: # αποθετηρίων (repos) που περιέχουν τουλάχιστον ένα αρχείο σε αυτή τη γλώσσα)

Για να είναι το γράφημα ευανάγνωστο, κρατήστε τις 20 επικρατέστερες γλώσσες.

**Συμβουλή:** https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays


In [0]:
%%bigquery --project $project_id q5b

SELECT language, COUNT(repo_name)
FROM
(SELECT language[OFFSET(0)].name AS language, repo_name
FROM `bigquery-public-data.github_repos.languages`
UNION DISTINCT
SELECT language[OFFSET(1)].name AS language, repo_name
FROM `bigquery-public-data.github_repos.languages`
UNION DISTINCT
SELECT language[OFFSET(2)].name AS language, repo_name
FROM `bigquery-public-data.github_repos.languages`
UNION DISTINCT
SELECT language[OFFSET(3)].name AS language, repo_name
FROM `bigquery-public-data.github_repos.languages`)
GROUP BY language

In [0]:
# YOUR PLOT CODE HERE

### c) Κατανομή μεγέθους αρχείων (1 μονάδα)
#### (x-άξονας: μέγεθος αρχείου, y-άξονας: # αρχείων με αυτό το μέγεθος) 

In [16]:
%%bigquery --project $project_id q5c

SELECT size, COUNT(id) AS Nr_of_files
FROM
(SELECT id, size
FROM `bigquery-public-data.github_repos.contents`)
GROUP BY size
ORDER BY Nr_of_files DESC LIMIT 4999


,size,Nr_of_files
0,399,639528
1,1507,380646
2,32,361356
3,129,320041
4,180,311033
5,192,292472
6,1181,275034
7,128,270350
8,178,252062
9,1177,218063


In [18]:
alt.Chart(q5c,height=600,width=600).mark_point().encode(
      x='size',
      y='Nr_of_files',
)

Chart({
  data:        size  Nr_of_files
  0       399       639528
  1      1507       380646
  2        32       361356
  3       129       320041
  4       180       311033
  5       192       292472
  6      1181       275034
  7       128       270350
  8       178       252062
  9      1177       218063
  10      130       214035
  11       95       208527
  12      263       202426
  13      162       201432
  14      175       192708
  15       98       191183
  16      179       187873
  17      181       185146
  18      176       181414
  19       72       176906
  20       24       176784
  21      191       173108
  22       40       172287
  23       50       165315
  24      219       164757
  25      183       159188
  26       60       158633
  27      218       157618
  28      212       157538
  29      177       157374
  ...     ...          ...
  4969   4750         9736
  4970   4755         9734
  4971   4817         9731
  4972   4861         9723
  4973   4731         9720
  4974   5014         9719
  4975   4805         9719
  4976  10360         9715
  4977   7393         9713
  4978   4727         9710
  4979   4797         9702
  4980   4738         9700
  4981   4890         9698
  4982   4746         9698
  4983   4859         9694
  4984   4785         9689
  4985   4799         9683
  4986   4753         9681
  4987   4887         9680
  4988   4882         9679
  4989   5124         9679
  4990   4749         9673
  4991   4910         9672
  4992   4789         9665
  4993   4765         9662
  4994   4721         9660
  4995   4758         9655
  4996   4766         9649
  4997   4969         9649
  4998   4883         9648
  
  [4999 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'size'
    }),
    y: Y({
      shorthand: 'Nr_of_files'
    })
  }),
  height: 600,
  mark: 'point',
  width: 600
})

### d) Κατανομή αρχείων που σχετίζονται με ένα αποθετήριο (repo) (1 μονάδα)
#### (x-άξονας: # αρχείων που σχετίζονται με ένα αποθετήριο (repo) , y-άξονας: # αποθετηρίων (repos) που σχετίζονται με τέτοιο πλήθος αρχείων)

In [26]:
%%bigquery --project $project_id q5d

SELECT COUNT(repo_name) AS Nr_of_repos, Nr_of_files
FROM
(SELECT repo_name, COUNT(id) AS Nr_of_files
FROM
(SELECT repo_name, id
FROM `bigquery-public-data.github_repos.files`)
GROUP BY repo_name)
GROUP BY Nr_of_files
ORDER BY Nr_of_repos DESC LIMIT 4999


,Nr_of_repos,Nr_of_files
0,165215,3
1,108295,2
2,107982,4
3,103796,5
4,98051,6
5,91019,7
6,87761,8
7,81600,9
8,76704,10
9,71307,11


In [31]:
alt.Chart(q5d,height=600,width=600).mark_point().encode(
      x='Nr_of_files',
      y='Nr_of_repos',
)

Chart({
  data:       Nr_of_repos  Nr_of_files
  0          165215            3
  1          108295            2
  2          107982            4
  3          103796            5
  4           98051            6
  5           91019            7
  6           87761            8
  7           81600            9
  8           76704           10
  9           71307           11
  10          68226           12
  11          61566           13
  12          58234           14
  13          54298           15
  14          51566           16
  15          48356           17
  16          44960           18
  17          42272           19
  18          39972           20
  19          37605           21
  20          35882           22
  21          33988           23
  22          31716           24
  23          30837           25
  24          29070           26
  25          28726           27
  26          27433           28
  27          25791           29
  28          24538           30
  29          23914           31
  ...           ...          ...
  4969           11         6024
  4970           11         4347
  4971           11         4208
  4972           11         4722
  4973           11         4517
  4974           11         5533
  4975           11         6612
  4976           11         4771
  4977           11         5834
  4978           11         5294
  4979           11        10680
  4980           11         6430
  4981           11         4360
  4982           11         4417
  4983           11         3375
  4984           11         5371
  4985           11         7003
  4986           11         5588
  4987           11         5102
  4988           11         4696
  4989           11         4511
  4990           11         6739
  4991           11         3860
  4992           11         4216
  4993           11         7981
  4994           11         3571
  4995           11         4459
  4996           11         6117
  4997           11         4282
  4998           11         4470
  
  [4999 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'Nr_of_files'
    }),
    y: Y({
      shorthand: 'Nr_of_repos'
    })
  }),
  height: 600,
  mark: 'point',
  width: 600
})

### e) Πλήθος αναθεωρήσεων (commits) κατά συντάκτη (author) και αναθεωρητή (committer) (3 μονάδες)
#### (x-άξονας: # commits, y-άξονας: # authors/committers με τόσα commits)

**Σημείωση**: στο εν λόγω διάγραμμα, σχεδιάστε τις καμπύλες για τους συντάκτες  (authors) και τους αναθεωρητές (committers) δίπλα - δίπλα για σύγκριση.



In [0]:
%%bigquery --project $project_id q5e_authors

# YOUR QUERY HERE

In [0]:
%%bigquery --project $project_id q5e_committers

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE - AUTHORS

In [0]:
# YOUR PLOT CODE HERE - COMMITTERS

### f) Σε μια παράγραφο (με λιγότερες από 100 λέξεις), περιγράψτε και αναλύστε τα διαγράμματα που δημιουργήσατε.  Ποιες ενδιαφέρουσες τάσεις παρατηρείτε στα δεδομένα; Προέκυψε κάτι που δεν ήταν αναμενόμενο; (2 μονάδες)

---
*Εισάγετε εδώ την ανάλυση των διαγραμμάτων σας*

---

## Ποια τα χαρακτηριστικά ενός καλού αποθετηρίου (repo)?

Με δεδομένο ότι έχουμε ενδιαφέροντα δεδομένα στη διάθεσή μας, ας προσπαθήσουμε να απαντήσουμε το ερώτημα: ποια τα χαρακτηριστικά ενός καλού αποθετηρίου (repo) του GitHub; Για το σκοπό μας "καλό" θεωρείται ένα αποθετήριο με μεγάλο αριθμό "παρατηρητών", δηλαδή ανθρώπων που παρακολουθούν το αποθετήριο για ενδεχόμενες αλλαγές.

Για αρχή, ας εξετάσουμε εάν κάποια από τα γνωρίσματα *που μόλις διερευνήσαμε* μας δίνουν καλές απαντήσεις.

## Ερώτηση 6: Ας χρησιμοποιήσουμε τα αποτελέσματα της προηγούμενης δουλειάς μας (10 μονάδες)

Φτιάξτε γραφικές παραστάσεις για τα ακόλουθα χαρακτηριστικά ενός αποθετηρίου (repo) σε σχέση με τον αριθμό παρατηρητών (watch count) του αποθετηρίου :

1.  Τύπος άδειας
2.  Γλώσσες (προγραμματισμού)
3.  Μέσο μέγεθος αρχείου στο αποθετήριο
4.  Πλήθος αρχείων αποθετηρίου
5.  Αριθμός ισχυρών αναθεωρητών ή συντακτών του αποθετηρίου ("power" committers / authors)


### a) Τύπος άδειας (1 μονάδα)

In [0]:
%%bigquery --project $project_id q6a

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### b) Γλώσσες (προγραμματισμού) (1 μονάδα)



In [0]:
%%bigquery --project $project_id q6b

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### c) Μέσο μέγεθος αρχείου στο αποθετήριο (1 μονάδα)
**Σημείωση:** Για την ερώτηση αυτή μπορείτε να χρησιμοποιήσετε τον πίνακα `github_repo_readme_contents_cs360` αντί του πίνακα με όλο το περιεχόμενο.


In [0]:
%%bigquery --project $project_id q6c

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### d) Πλήθος αρχείων ενός αποθετηρίου (1 μονάδα)

In [0]:
%%bigquery --project $project_id q6d

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### e) Αριθμός ισχυρών αναθεωρητών ή συντακτών ενός αποθετηρίου ("power" committers / authors) (3 μονάδες)

 **Ορισμός:**  "ισχυρός" αναθεωρητής ή συντάκτης είναι ένας λογαριασμός (account) μέσω του οποίου έχουν γίνει **τουλάχιστον 1,000 αναθεωρήσεις** (commits) σε αναθεωρήσεις ή συντάξεις (commit/author).

In [0]:
%%bigquery --project $project_id q6e_power_committers

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

In [0]:
%%bigquery --project $project_id q6e_power_authors

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### f) Από όσα μελετήσαμε, υπάρχουν γνωρίσματα και ποια είναι αυτά που έχουν τη μεγαλύτερη συσχέτιση με τα αποθετήρια (repos) με υψηλό αριθμό παρατηρητών; Είναι λογικοφανής η απάντησή σας ή μοιάζει να αντιβαίνει τη διαίσθησή σας; Δώστε την απάντησή σας σε μια παράγραφο, όχι μεγαλύτερη των 200 λέξεων. Αναφερθείτε στις γραφικές παραστάσεις που φτιάξατε. (3 μονάδες)

---
*Εισάγετε εδώ την απάντησή σας*

---